In [2]:

from jetbot import Robot
import traitlets
import ipywidgets.widgets as widgets
from jetbot import Camera
from jetbot import bgr8_to_jpeg
import cv2
import numpy as np
import time



Thresholding is based on lab environment whee the tape was on grey background.

In [9]:
# Initialize camera
camera = Camera.instance()
image = widgets.Image(format='jpeg', width=640, height=480)
Blackline_image = widgets.Image(format='jpeg', width=640, height=480)

def process_frame(change): 
    Greendetected =False
    Blackdetected=False
    
    frame = change['new'] 
    roi=frame[50:150, 0:640] 
    Blackline=cv2.inRange(roi, (15,15,15), (100,100,100)) 
    Greensign=cv2.inRange(roi, (15,50,15), (110,250,110))
    
    
    
    kernel=np.ones((5,5), np.uint8) 
    Blackline=cv2.erode(Blackline,kernel,iterations=5) 
    Blackline=cv2.dilate(Blackline,kernel,iterations=6) 
    Greensign=cv2.erode(Greensign,kernel,iterations=5) 
    Greensign=cv2.dilate(Greensign,kernel,iterations=6) 
    
    
    contours_blk,hierarchy_blk = cv2.findContours(Blackline.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE) 
    contours_green,hierarchy_green = cv2.findContours(Greensign.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE) 
    
    
    if len(contours_green)>0:
        Greendetected = True
        x_green,y_green,w_green,h_green=cv2.boundingRect(contours_green[0]) 
        centerx_green=x_green+(w_green//2)
        cv2.line(frame, (centerx_green, 50), (centerx_green  , 150), (0, 0, 255), 1) 

    
    
    # cv2.drawContours(roi,contours,-1,(0,255,0),1) 
    if len(contours_blk)>0: 
        Blackdetected=True
        largest = max(contours_blk, key=cv2.contourArea)
        x_blk,y_blk,w_blk,h_blk=cv2.boundingRect(largest) 
      #  cv2.rectangle(roi, (x,y), (x+w,y+h),(0,0,255),1) 
        centerx_blk=x_blk+(w_blk//2)
        cv2.line(roi, (centerx_blk, 50), (centerx_blk  , 150), (255, 0, 0), 1) 
        
        
    if Greendetected and Blackdetected:
        if centerx_green>centerx_blk:
            cv2.putText(frame, "Turn Right", (50,100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0),1)
        else:
            cv2.putText(frame, "Turn Left", (50,100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0),1)
    
    if Blackdetected:
        setpoint=100
        deviation=centerx_blk-setpoint
        centertext="Deviation = "+str(deviation)
        cv2.putText(frame, centertext, (70,120), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0),1)
        
    image.value = bgr8_to_jpeg(frame) 
    Blackline_bgr=cv2.cvtColor(Blackline, cv2.COLOR_GRAY2BGR) 
    Blackline_image.value=bgr8_to_jpeg(Blackline_bgr) 

display(image) 
camera.observe(process_frame, names='value')

Image(value=b'', format='jpeg', height='480', width='640')

In [10]:
camera.stop()

